<a href="https://colab.research.google.com/github/ericwarren9/ST-590/blob/main/Warren_Grant_ST_590_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ST 590 Project 1 By: Eric Warren and David Grant

The goal of this project is to use a number of .csv files that contain information from the census bureau surveys. This data is a
little older (2010 is the newest data). Our first goal will be to read in one of these .csv files and parse the data using functions we've written. Then we'll combine some parsed data and deal with it from there.

## Introduction

In this report, we are going to look at enrollment data and how to go from a raw data file (in this case a CSV) and turn it into readable information that we can use for analysis. Along the way, we will be making functions that will make the repetition of the same tasks easier for us as we progress with using multiple files. After getting this readable data, we will move on to make two competing models (one that is a simple linear regression and the other that is a multiple linear regression model) that will be looked at for trying to predict our enrollment values. We will end the report by discussing the [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error) values we get from doing cross-validation with our models, with the lowest mean squared error highlighting the model we should choose to use for prediction.

## Part 1: Data Processing

### First Steps

First, read in the data set available here: [https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv](https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv)

In [1]:
# Import library to read in data
import pandas as pd

# Read in the data
edu_df_a = pd.read_csv("https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv", sep = ",")

# Show the first couple of observations to get idea of data
edu_df_a.head()

,Area_name,STCOU,EDU010187F,EDU010187D,EDU010187N1,EDU010187N2,EDU010188F,EDU010188D,EDU010188N1,EDU010188N2,...,EDU010194N1,EDU010194N2,EDU010195F,EDU010195D,EDU010195N1,EDU010195N2,EDU010196F,EDU010196D,EDU010196N1,EDU010196N2
0,UNITED STATES,0,0,40024299,0,0,0,39967624,0,0,...,0,0,0,43993459,0,0,0,44715737,0,0
1,ALABAMA,1000,0,733735,0,0,0,728234,0,0,...,0,0,0,727989,0,0,0,736825,0,0
2,"Autauga, AL",1001,0,6829,0,0,0,6900,0,0,...,0,0,0,7568,0,0,0,7834,0,0
3,"Baldwin, AL",1003,0,16417,0,0,0,16465,0,0,...,0,0,0,19961,0,0,0,20699,0,0
4,"Barbour, AL",1005,0,5071,0,0,0,5098,0,0,...,0,0,0,5017,0,0,0,5053,0,0


Note the format of the data is kind of weird. There is a column for `Area_name` (US, NC, or a county), a column called `STCOU`, and then four columns corresponding to each question on the survey. This survey was
about school enrollment. Now let us process this data!

Select only the following columns:
- Area_name
- STCOU
- Any column that ends in “D”
- To do the above, use the `.loc[]` method. Note that the `.str.endswith()`
method can be used on the column names for the columns that end with “D”

In [2]:
# Get list of names that end in 'D'
lst = list(edu_df_a.columns[edu_df_a.columns.str.endswith("D")])

# Combine with columns we need to subset by
name_lst = ['Area_name', 'STCOU'] + lst

# Keep only columns we need in the named list
selected_edu_a = edu_df_a.loc[:, name_lst]

# Return the first couple of rows to show this worked
selected_edu_a.head()

,Area_name,STCOU,EDU010187D,EDU010188D,EDU010189D,EDU010190D,EDU010191D,EDU010192D,EDU010193D,EDU010194D,EDU010195D,EDU010196D
0,UNITED STATES,0,40024299,39967624,40317775,40737600,41385442,42088151,42724710,43369917,43993459,44715737
1,ALABAMA,1000,733735,728234,730048,728252,725541,726150,728014,730509,727989,736825
2,"Autauga, AL",1001,6829,6900,6920,6847,7008,7137,7152,7381,7568,7834
3,"Baldwin, AL",1003,16417,16465,16799,17054,17479,17983,18735,19384,19961,20699
4,"Barbour, AL",1005,5071,5098,5068,5156,5173,5252,5135,5111,5017,5053


The data we have is in wide format. That is, there are multiple observations in a given row (each column that ends in “D” corresponds to a particular year's measurement). We want to convert the data into long format where each row has only one enrollment value for that Area_name. Do this operation using the `pd.melt()` function.

In [3]:
# Change to long data
edu_a_long_df = selected_edu_a.melt(id_vars = ['Area_name', 'STCOU'], value_name = "enrollment")

# Show the first couple of rows to show it worked
edu_a_long_df.head()

,Area_name,STCOU,variable,enrollment
0,UNITED STATES,0,EDU010187D,40024299
1,ALABAMA,1000,EDU010187D,733735
2,"Autauga, AL",1001,EDU010187D,6829
3,"Baldwin, AL",1003,EDU010187D,16417
4,"Barbour, AL",1005,EDU010187D,5071


One of the new columns should now correspond to the old column names that ended with a “D”. The first three characters of the former column names represent the survey and the next four characters
represent the type of value you have from that survey. The last two characters prior to the “D” represent
the year of the measurement. For this part:

- Create a loop that loops over the rows of the data frame
- At each iteration,
  - parse the string from your new column in order to pull out the year and convert it into a numeric value such as 1997 or 2002. Add this new year variable to your data frame. Note: the data set above only has data from the 1900's but the next data set we read in has data from the 2000's. Handle that appropriately!
  - grab the first three characters and following four digits to create a new variable representing which measurement was grabbed. Add this new measurement variable to your data frame as well.
- Drop the original column name variable.

In [4]:
# import numpy to do calculation
import numpy as np

#Use for loop to get all of this part
edu_a_df_updated = edu_a_long_df # Make copy of data frame to do manipulations
yr = [] # Initialize year data
measure = [] # Initialize measurement data

for i in edu_a_df_updated.index:

  # Get the year
  if pd.to_numeric(edu_a_df_updated.variable[i][-3:-1]) <= 24:
    yr.append(pd.to_numeric('20' + edu_a_df_updated.variable[i][-3:-1]))
  else:
    yr.append(pd.to_numeric('19' + edu_a_df_updated.variable[i][-3:-1]))

  # Get the measurement
  measure.append(edu_a_df_updated.variable[i][:7])

# Create new columns
edu_a_df_updated['Year'] = yr
edu_a_df_updated['measurement'] = measure

# Drop the variable column
edu_a_df_updated = edu_a_df_updated.drop(columns = 'variable')

# How to do this step without for loop
# edu_a_df_updated = edu_a_long_df
# edu_a_df_updated['Year'] = np.where(pd.to_numeric(edu_a_df_updated.variable.str[-3:-1], errors = 'coerce') <= 24, \
#                                    pd.to_numeric("20" + edu_a_df_updated.variable.str[-3:-1]), \
#                                    pd.to_numeric("19" + edu_a_df_updated.variable.str[-3:-1]))
# edu_a_df_updated = edu_a_df_updated.assign(measurement = edu_a_df_updated.variable.str[:7])
# edu_a_df_updated = edu_a_df_updated.drop(columns= 'variable')

# Check to make sure we get what we are asked
edu_a_df_updated

,Area_name,STCOU,enrollment,Year,measurement
0,UNITED STATES,0,40024299,1987,EDU0101
1,ALABAMA,1000,733735,1987,EDU0101
2,"Autauga, AL",1001,6829,1987,EDU0101
3,"Baldwin, AL",1003,16417,1987,EDU0101
4,"Barbour, AL",1005,5071,1987,EDU0101
...,...,...,...,...,...
31975,"Sweetwater, WY",56037,9599,1996,EDU0101
31976,"Teton, WY",56039,2226,1996,EDU0101
31977,"Uinta, WY",56041,5750,1996,EDU0101
31978,"Washakie, WY",56043,1900,1996,EDU0101


Split the data frame into two data frames:
- one data frame should contain only non-county data
- the other should contain only county level data

Note that all county measurements have the format “County Name, DD” where “DD” represents the state. Use the .apply() method with a lambda function to create an indexing vector you use to do the subsetting. np.logical_not() comes in handy!

In [5]:
# Get index of what has county data
index = edu_a_df_updated.Area_name.apply(lambda x: np.logical_not("," in x))

# Get data frame of non-county data
non_county_edu_a = edu_a_df_updated[index]

# Get data frame of county data
county_edu_a = edu_a_df_updated[~index]

For the county level data frame, create a new variable that describes which state one of these county measurements corresponds to (the two digit abbreviation is fine!).

In [6]:
# Get the state
county_edu_a = county_edu_a.assign(state = county_edu_a.Area_name.str[-2:])

# Return first couple of rows to show state column works
county_edu_a.head()

,Area_name,STCOU,enrollment,Year,measurement,state
2,"Autauga, AL",1001,6829,1987,EDU0101,AL
3,"Baldwin, AL",1003,16417,1987,EDU0101,AL
4,"Barbour, AL",1005,5071,1987,EDU0101,AL
5,"Bibb, AL",1007,3557,1987,EDU0101,AL
6,"Blount, AL",1009,7319,1987,EDU0101,AL


For the non-county level data frame, create a new variable called division corresponding to the state's classification of division [here](https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States) (the Census Bureau-designated regions and divisions). If the row
corresponds to a non-state (i.e. UNITED STATES), return ERROR for the division. The code for this part will not be a ton of fun to write! Write a function with basic if/elif for a single value of Area_name.
Then, use `np.vectorize()` to make it work for a full vector of values.

In [7]:
# Make function
def get_division(x):
  """
  Will give us the division of the state
  """

  if x in list(map(lambda x: x.upper(), ["Connecticut", "Maine", "Massachusetts", "New Hampshire", "Rhode Island", "Vermont"])):
    return "1"
  elif x in list(map(lambda x: x.upper(), ["New Jersey", "New York", "Pennsylvania"])):
    return "2"
  elif x in list(map(lambda x: x.upper(), ["Illinois", "Indiana", "Michigan", "Ohio", "Wisconsin"])):
    return "3"
  elif x in list(map(lambda x: x.upper(), ["Iowa", "Kansas", "Minnesota", "Missouri", "Nebraska", "North Dakota", "South Dakota"])):
    return "4"
  elif x in list(map(lambda x: x.upper(), ["Delaware", "Florida", "Georgia", "Maryland", "North Carolina", "South Carolina", "Virginia", "District of Columbia", "West Virginia"])):
    return "5"
  elif x in list(map(lambda x: x.upper(), ["Alabama", "Kentucky", "Mississippi", "Tennessee"])):
    return "6"
  elif x in list(map(lambda x: x.upper(), ["Arkansas", "Louisiana", "Oklahoma", "Texas"])):
    return "7"
  elif x in list(map(lambda x: x.upper(), ["Arizona", "Colorado", "Idaho", "Montana", "Nevada", "New Mexico", "Utah", "Wyoming"])):
    return "8"
  elif x in list(map(lambda x: x.upper(), ["Alaska", "California", "Hawaii", "Oregon", "Washington"])):
    return "9"
  else:
    return "ERROR"

# Now use np.vectorize to get new function to use
vfunc = np.vectorize(get_division)

# Create the new division column
non_county_edu_a = non_county_edu_a.assign(division = vfunc(non_county_edu_a.Area_name))

# Show the first couple of rows to see if it worked
non_county_edu_a.head()

,Area_name,STCOU,enrollment,Year,measurement,division
0,UNITED STATES,0,40024299,1987,EDU0101,ERROR
1,ALABAMA,1000,733735,1987,EDU0101,6
69,ALASKA,2000,102872,1987,EDU0101,9
99,ARIZONA,4000,609411,1987,EDU0101,8
115,ARKANSAS,5000,429260,1987,EDU0101,7


### Requirements

Now we want to repeat the above process for the 2nd component of the data set. This is available at the
link below.
- [https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv](https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv)

Rather than copying and pasting a bunch of stuff and changing things here and there, we want to write functions that do the above pieces and one function that we can call to do it all!

Write one function that takes care of steps 1 & 2 above. Give an optional argument (that is it has a default value) that allows the user to specify the name of the column representing the value ('enrollment' for these two data sets).

In [8]:
def get_data(data_file, col_name = "enrollment"):
  """
  This function will allow us to input a raw data file to get it in a form we can use.
  The col_name argument will allow the user to select the name of the new column name we come up with.
  """

  # First let us take care of step 1
  import pandas as pd # Import library to read in data

  df = pd.read_csv(data_file, sep = ",") # Read in the data
  lst = list(df.columns[df.columns.str.endswith("D")]) # Get list of names that end in 'D'
  name_lst = ['Area_name', 'STCOU'] + lst # Combine with columns we need to subset by
  selected_df = df.loc[:, name_lst] # Keep only columns we need in the named list

  # Now take care of step 2
  long_df = selected_df.melt(id_vars = ['Area_name', 'STCOU'], value_name = col_name)

  # return our data set
  return long_df


# Example of function being used
test = get_data("https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv")
test.head()

,Area_name,STCOU,variable,enrollment
0,UNITED STATES,0,EDU010197D,44534459
1,ALABAMA,1000,EDU010197D,737386
2,"Autauga, AL",1001,EDU010197D,8099
3,"Baldwin, AL",1003,EDU010197D,21410
4,"Barbour, AL",1005,EDU010197D,5100


Write another function that takes in the output of step 2 and does step 3 above.

In [9]:
def get_survey_info(data):
  """
  Here we are going to input our data and make sure get it updated with the survey information like year and survey type.
  """

  # Use for loop to get all of this part
  year = [] # Initialize year data
  measure = [] # Initialize measurement data

  for i in data.index:

    # Get the year
    if pd.to_numeric(data.variable[i][-3:-1], errors = 'coerce') <= 24:
      year.append(pd.to_numeric('20' + data.variable[i][-3:-1]))
    else:
      year.append(pd.to_numeric('19' + data.variable[i][-3:-1]))

    # Get the measurement
    measure.append(data.variable[i][:7])

  # Add new columns
  data['Year'] = year
  data['measurement'] = measure

  # Drop the variable column
  data = data.drop(columns = 'variable')

  return data # Return our data

# Show how function works
test2 = get_survey_info(test)
test2

,Area_name,STCOU,enrollment,Year,measurement
0,UNITED STATES,0,44534459,1997,EDU0101
1,ALABAMA,1000,737386,1997,EDU0101
2,"Autauga, AL",1001,8099,1997,EDU0101
3,"Baldwin, AL",1003,21410,1997,EDU0101
4,"Barbour, AL",1005,5100,1997,EDU0101
...,...,...,...,...,...
31975,"Sweetwater, WY",56037,6964,2006,EDU0152
31976,"Teton, WY",56039,2264,2006,EDU0152
31977,"Uinta, WY",56041,4298,2006,EDU0152
31978,"Washakie, WY",56043,1410,2006,EDU0152


Write a function to do step 5.

In [10]:
def get_state(data):
  """
  This function will allow you to create a new column that gets the state of county.
  """

  data = data.assign(state = data.Area_name.str[-2:])  # Get the state information
  return data # Return data

Write a function to do step 6.

In [11]:
def add_division(data):
  """
  This function will return a new column of the state data that gets the division the state is in for US Census.
  """

  import numpy as np # Make sure we use numpy for vectorizing

  vfunc = np.vectorize(get_division) # Now use np.vectorize to get new function to use
  data = data.assign(division = vfunc(data.Area_name)) # Create the new division column
  return data # Return data


Write another function that takes in the output from step 3 and creates the two data frames in step 4, calls the above two functions (to perform steps 5 and 6), and returns two final data frames.

In [12]:
def separate_data(data):
  """
  This function separates our data into county and non-county data.
  Then we will return the extra columns and analysis for each data set
  """

  import numpy as np # Needed for calculations

  # Do step 4
  index = data.Area_name.apply(lambda x: np.logical_not("," in x))  # Get index of what has county data
  non_county_data = data[index] # Get data frame of non-county data
  county_data = data[~index] # Get data frame of county data

  # Combine step 5 function for county data
  county_data_state_info = get_state(county_data)

  # Combine step 6 function for non-county data
  non_county_data_division_info = add_division(non_county_data)

  # Return both data sets
  return county_data_state_info, non_county_data_division_info

# Test to show how it works
test3 = separate_data(test2)
test3

(            Area_name  STCOU  enrollment  Year measurement state
 2         Autauga, AL   1001        8099  1997     EDU0101    AL
 3         Baldwin, AL   1003       21410  1997     EDU0101    AL
 4         Barbour, AL   1005        5100  1997     EDU0101    AL
 5            Bibb, AL   1007        3717  1997     EDU0101    AL
 6          Blount, AL   1009        7816  1997     EDU0101    AL
 ...               ...    ...         ...   ...         ...   ...
 31975  Sweetwater, WY  56037        6964  2006     EDU0152    WY
 31976       Teton, WY  56039        2264  2006     EDU0152    WY
 31977       Uinta, WY  56041        4298  2006     EDU0152    WY
 31978    Washakie, WY  56043        1410  2006     EDU0152    WY
 31979      Weston, WY  56045        1076  2006     EDU0152    WY
 
 [31450 rows x 6 columns],
            Area_name  STCOU  enrollment  Year measurement division
 0      UNITED STATES      0    44534459  1997     EDU0101    ERROR
 1            ALABAMA   1000      737386  1

Note we can see our two data frames saved as a tuple.

Now last thing, put it all into one function call! This is called creating a wrapper function. Create a function that takes in the URL of a .csv file in this format and the optional argument for the variable name, calls the functions you wrote above, and then returns the two data frames in a list.

In [13]:
def raw_data_to_datasets(data_file, col_name = "enrollment"):
  """
  This function changes our raw data into a list of data frames we can use.
  """

  # Use first function to make raw data usable.
  df = get_data(data_file = data_file, col_name = col_name)

  # Now use section function to get survey information
  df2 = get_survey_info(df)

  # Lastly get list of data frames
  df3 = separate_data(df2)

  # Return data frames as list
  return list(df3)

# Show this works on data file
test4 = raw_data_to_datasets("https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv")
test4


[            Area_name  STCOU  enrollment  Year measurement state
 2         Autauga, AL   1001        8099  1997     EDU0101    AL
 3         Baldwin, AL   1003       21410  1997     EDU0101    AL
 4         Barbour, AL   1005        5100  1997     EDU0101    AL
 5            Bibb, AL   1007        3717  1997     EDU0101    AL
 6          Blount, AL   1009        7816  1997     EDU0101    AL
 ...               ...    ...         ...   ...         ...   ...
 31975  Sweetwater, WY  56037        6964  2006     EDU0152    WY
 31976       Teton, WY  56039        2264  2006     EDU0152    WY
 31977       Uinta, WY  56041        4298  2006     EDU0152    WY
 31978    Washakie, WY  56043        1410  2006     EDU0152    WY
 31979      Weston, WY  56045        1076  2006     EDU0152    WY
 
 [31450 rows x 6 columns],
            Area_name  STCOU  enrollment  Year measurement division
 0      UNITED STATES      0    44534459  1997     EDU0101    ERROR
 1            ALABAMA   1000      737386  1

As we can see our function works to give us a list of 2 dataframes with 1 being for the county data and the other being for non-county data.

### Call it and Combine Your Data

Call the function you made two times to read in and parse the two .csv files mentioned so far. Be sure to call the new value column the same in both function calls.

In [14]:
edu_df_1 = raw_data_to_datasets("https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv")
edu_df_2 = raw_data_to_datasets("https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv")

Now we want to join the two county level data sets and the two state level data sets. Write a function that takes in unlimited positional arguments. When you call the function these arguments will be the results of
calls to your wrapper function (so each argument will be a list with the two data sets in it).

- Within the function itself, use `map()` and a `lambda` function to obtain just the county level data for every argument. Then use the `reduce()` function from the functools module with a `lambda` function that calls `pd.concat()`.
- Repeat for the non county level data.
- Put the two combined data sets into a list and return it (so it will have the same format as the inputs)!


Call this function to combine the two data objects into one object (that has two data frames: the combined county level data and the combined non-county level data).

In [15]:
# import functools for reduce function
from functools import reduce

# Make function -- finish this
def combine_data(*args, col_name = "enrollment"):
  """
  This function will turn our unlimited arguments (data files) into a returned list of county and non-county data.
  """

  # Get all data
  data = list(map(lambda x: raw_data_to_datasets(x, col_name = col_name), args))

  # Get county data
  county_list = list(map(lambda x: x[0], data))
  county_df = reduce(lambda x, y: pd.concat([x, y]), county_list)
  #county_df = pd.concat(county_list, axis = 0, ignore_index = True) # This works but not how instructions were

  # Get non-county data
  non_county_list = list(map(lambda x:x[1], data))
  non_county_df = reduce(lambda x, y: pd.concat([x, y]), non_county_list)
  # non_county_df = pd.concat(non_county_list, axis = 0, ignore_index = True) # This works but not how instructions were

  # Return this list of dfs
  return list((county_df, non_county_df))

# Show our function works for our data
edu_data = combine_data("https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv", "https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv")
edu_data

[            Area_name  STCOU  enrollment  Year measurement state
 2         Autauga, AL   1001        6829  1987     EDU0101    AL
 3         Baldwin, AL   1003       16417  1987     EDU0101    AL
 4         Barbour, AL   1005        5071  1987     EDU0101    AL
 5            Bibb, AL   1007        3557  1987     EDU0101    AL
 6          Blount, AL   1009        7319  1987     EDU0101    AL
 ...               ...    ...         ...   ...         ...   ...
 31975  Sweetwater, WY  56037        6964  2006     EDU0152    WY
 31976       Teton, WY  56039        2264  2006     EDU0152    WY
 31977       Uinta, WY  56041        4298  2006     EDU0152    WY
 31978    Washakie, WY  56043        1410  2006     EDU0152    WY
 31979      Weston, WY  56045        1076  2006     EDU0152    WY
 
 [62900 rows x 6 columns],
            Area_name  STCOU  enrollment  Year measurement division
 0      UNITED STATES      0    40024299  1987     EDU0101    ERROR
 1            ALABAMA   1000      733735  1

## Double Check it Generalizes

Read in another similar data set and apply our functions!

-Run your data processing and combination functions on the four data sets at URLs given below:
 - [https://www4.stat.ncsu.edu/~online/datasets/PST01a.csv](https://www4.stat.ncsu.edu/~online/datasets/PST01a.csv)
 - [https://www4.stat.ncsu.edu/~online/datasets/PST01b.csv](https://www4.stat.ncsu.edu/~online/datasets/PST01b.csv)
 - [https://www4.stat.ncsu.edu/~online/datasets/PST01c.csv](https://www4.stat.ncsu.edu/~online/datasets/PST01c.csv)
 - [https://www4.stat.ncsu.edu/~online/datasets/PST01d.csv](https://www4.stat.ncsu.edu/~online/datasets/PST01d.csv)

In [16]:
# Call function on our new data
pst_data = combine_data("https://www4.stat.ncsu.edu/~online/datasets/PST01a.csv", \
                        "https://www4.stat.ncsu.edu/~online/datasets/PST01b.csv", \
                        "https://www4.stat.ncsu.edu/~online/datasets/PST01c.csv", \
                        "https://www4.stat.ncsu.edu/~online/datasets/PST01d.csv", \
                        col_name = "default_col_name") # Shows you can still change the column name if you do not like the "enrollment" name

# Show how it looks
pst_data

[            Area_name  STCOU  default_col_name  Year measurement state
 2         Autauga, AL   1001             25508  1971     PST0151    AL
 3         Baldwin, AL   1003             60141  1971     PST0151    AL
 4         Barbour, AL   1005             23092  1971     PST0151    AL
 5            Bibb, AL   1007             13919  1971     PST0151    AL
 6          Blount, AL   1009             27817  1971     PST0151    AL
 ...               ...    ...               ...   ...         ...   ...
 31975  Sweetwater, WY  56037             41226  2009     PST0452    WY
 31976       Teton, WY  56039             20710  2009     PST0452    WY
 31977       Uinta, WY  56041             20927  2009     PST0452    WY
 31978    Washakie, WY  56043              7911  2009     PST0452    WY
 31979      Weston, WY  56045              7009  2009     PST0452    WY
 
 [125800 rows x 6 columns],
            Area_name  STCOU  default_col_name  Year measurement division
 0      UNITED STATES      0    

## Cross-Validation

For the last part of the project we'll fit two linear regression models and judge them using cross-validation (no training/test split, we'll just use CV). However, we won't be able to use standard cross-validation. We'll
write our own function to do it!

### Subset of Data

Use the `enrollment` data sets from earlier.

- Subset the list object so that we’re only looking at the non county level data.
- Remove any rows where the division variable is ERROR and select only the year, division, and enrollment variables (or whatever you called the last one!).

In [17]:
# Get the edu non-county data
edu_non_county = edu_data[1]

# Get data where division is not error
edu_non_county_filtered = edu_non_county.loc[edu_non_county['division'] != "ERROR", ['Year', 'division', 'enrollment']]

# Show our new data we are using
edu_non_county_filtered

,Year,division,enrollment
1,1987,6,733735
69,1987,9,102872
99,1987,8,609411
115,1987,7,429260
191,1987,9,4621126
...,...,...,...
31650,2006,5,1220440
31787,2006,9,1026774
31827,2006,5,281938
31883,2006,3,876700


### Two Models Under Consideration

We will use two competing models:

- A SLR model using year to predict enrollment
- An MLR model using year and division to predict enrollment
We will need to create dummy variables for the division variable as we did in the notes and add them to the data frame. When adding these columns to the data frame, you shouldn't keep all 9 variables (recall the
last column is redundant given all the others). Be sure to leave one of the indicator columns off!

Feel free to fit models in the step (but do not have to).

In [18]:
# Make dummy variable data frame
dummy_edu_df = pd.get_dummies(data = edu_non_county_filtered['division'])

# Make combined data frame for analysis, iloc drops last column
edu_non_county_final_df = pd.concat([edu_non_county_filtered, dummy_edu_df], axis = 1).iloc[:, :-1]

# Show what our data now looks like before analysis
edu_non_county_final_df

,Year,division,enrollment,1,2,3,4,5,6,7,8
1,1987,6,733735,0,0,0,0,0,1,0,0
69,1987,9,102872,0,0,0,0,0,0,0,0
99,1987,8,609411,0,0,0,0,0,0,0,1
115,1987,7,429260,0,0,0,0,0,0,1,0
191,1987,9,4621126,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
31650,2006,5,1220440,0,0,0,0,1,0,0,0
31787,2006,9,1026774,0,0,0,0,0,0,0,0
31827,2006,5,281938,0,0,0,0,1,0,0,0
31883,2006,3,876700,0,0,1,0,0,0,0,0


Please note we can see how there is no column for division **9**. This is because if we get zero for the first 8 dummy variables then it is implied the division is 9 (the last possible division). We can also make what our models should look like below. Note the code is commented out but this is what we will do in our proceeding step with cross validation.

In [19]:
# import modules and functions
#from sklearn import linear_model, metrics

# Make SLR model, uncomment below to see initial model
# reg1 = linear_model.LinearRegression()
# reg1.fit(edu_non_county_final_df['Year'].values.reshape(-1,1), edu_non_county_final_df['enrollment'])
# print(reg1.intercept_, reg1.coef_)

# Make MLR model, uncomment below to see initial model
# reg2 = linear_model.LinearRegression()
# reg2.fit(edu_non_county_final_df.loc[:, ~edu_non_county_final_df.columns.isin(['division', 'enrollment'])], \
#         edu_non_county_final_df['enrollment'])
# print(reg2.intercept_, reg2.coef_)

### Cross Validation

We want to see how well these two competing models do at predicting. However, we can't use the usual cross-validation because our data is over time (year). Instead, what we want to do is train the model/judge it sequentially.

1. Use the first three years of data to fit the model. Use that model to predict the fourth year. Calculate
the MSE for those predictions.
2. Use the first four years of data to fit the model. Use that model to predict the fifth year. Calculate
the MSE for those predictions.
3. Repeat until you predict for the last year.
4. Sum up the MSE values to get an overall MSE for the model!

Write a function to do the above given a particular X, y, and starting `year`.

Some guidelines and helpful hints for doing this step:

- First write a function to get the MSE for one step of the above only (don't worry about combining things yet).
  - Have this function take in a data frame of predictors X (this will be used in the `.fit()` method of a `LinearRegression()` object), a 1D response y, and a last_year argument.
  - Use the last_year argument to subset the data into a training X and y and a testing X and y. Have your training set include all years up to and including last_year and your test set just include the year last_year + 1.
  - Do the model fitting and predictions. Return the mean squared error
- That will act as a helper function for our function that find the CV error.
- Now write a function to obtain the CV value over all the years (other than the initial training block)
- Have this function take in X, y (both as above), and a first_year argument.
  - If the first_year is less than 1989, have the function raise an error and return a message
  - Initialize an MSE value at 0
  - If the not, use a loop from first_year to the last year in the data set (find that value programmatically)
    - Within the loop, use your previous function and augmented assignment to add the MSE for the given year
  - Return the MSE

First let us make our MSE formula for the case calling the last year.

In [20]:
def calculate_mse(X, y, last_year):
  """
  This calculates the MSE for our modeled data
  """

  from sklearn import linear_model
  import sklearn.metrics as metrics

  # What type of model should we do
  if isinstance(X, pd.Series):
    # Subset data
    trainX = X[X <= last_year]
    testX = X[X == last_year + 1]
    trainY = y[:len(trainX.index)]
    testY = y[len(trainX.index):len(trainX.index) + len(testX.index)]
    reg = linear_model.LinearRegression()
    reg.fit(trainX.values.reshape(-1,1), trainY.values)
    pred = reg.predict(testX.values.reshape(-1,1))
    mse = metrics.mean_squared_error(testY.values, pred)
  else:
    # Subset data
    trainX = X.loc[X['Year'] < last_year]
    testX = X.loc[X['Year'] == last_year]
    trainY = y[:len(trainX.index)]
    testY = y[len(trainX.index):len(trainX.index) + len(testX.index)]
    reg2 = linear_model.LinearRegression()
    reg2.fit(trainX, trainY.values)
    pred2 = reg2.predict(testX)
    mse = metrics.mean_squared_error(testY.values, pred2)
  return mse

Show that it works for SLR case.

In [21]:
calculate_mse(X = edu_non_county_final_df['Year'], y = edu_non_county_final_df['enrollment'], last_year = 1994)

905563241569.4473

Show it works for MLR case.

In [22]:
calculate_mse(X = edu_non_county_final_df.loc[:, ~edu_non_county_final_df.columns.isin(['division', 'enrollment'])], y = edu_non_county_final_df['enrollment'], last_year = 1994)

640487924956.1287

Now create function to take cross validation of all years.

In [23]:
def pred_error(X, y, first_year):
  """
  This function will get the first 3 years starting with the starting year and then return the MSE value with the next year after that and will go until all years exhausted
  """
  # Raise error if starting_year is before 1989
  if first_year < 1989:
    raise ValueError("Input for starting_year must be 1989 or after")

  # Get iteration of MSE values
  if isinstance(X, pd.Series):
    max_year = X.max()
  else:
    max_year = X.Year.max()

  # Added this not required but want at least 3 years of training data and 1 testing
  if max_year - 3 < first_year: # Do minus 3 because years 0 through 2 go to training data and year 3 goes to test.
    raise ValueError("Please select a starting_year where you will have at least 4 different years of data. For example if the last year of the data is 2006 then select at the latest 2003.")

  # Do loop to get total MSE value
  last_year = first_year + 2 # Show last year as function of first year; also note that we want years 0, 1, 2 for the first 3 years; why we add 2 and not 3 here
  mse = 0 # Intialize MSE
  while last_year <= max_year - 1:
    mse += calculate_mse(X, y, last_year)
    last_year += 1

  # Return the MSE values added up
  return mse

Now run our function using the SLR model.

In [24]:
slr_mse = pred_error(X = edu_non_county_final_df['Year'], y = edu_non_county_final_df['enrollment'], first_year = 1991)

slr_mse

14227804635389.143

Repeat using the MLR model.

In [25]:
mlr_mse = pred_error(X = edu_non_county_final_df.loc[:, ~edu_non_county_final_df.columns.isin(['division', 'enrollment'])], y = edu_non_county_final_df['enrollment'], first_year = 1991)

mlr_mse

10412585840543.791

Show error message if year is too early.

In [26]:
pred_error(X = edu_non_county_final_df['Year'], y = edu_non_county_final_df['enrollment'], first_year = 1988)

ValueError: Input for starting_year must be 1989 or after

Also show another message if the year is too late so we do not get at least 3 years of training data.

In [27]:
pred_error(X = edu_non_county_final_df['Year'], y = edu_non_county_final_df['enrollment'], first_year = 2004)

ValueError: Please select a starting_year where you will have at least 4 different years of data. For example if the last year of the data is 2006 then select at the latest 2003.

Discuss the MSE values we see. First we will see which model has a lower MSE based on our example above for both models. We will see if our Multiple Linear Regression model is better (by having a lower MSE) than our Simple Linear Regression model. Note this only looks at when 1991 is the chosen selected year but this trend continues for the years selected.

In [28]:
mlr_mse < slr_mse

True

As we can see the Multiple Linear Regression model does a better job in making predictions than our Simple Linear Regression model of just Year. This makes sense conceptionally since we know that different divisions of states in the country have different amount of citizens (and thus different amount of enrollment levels). That is why when we add the dummy variable of the predictor of division, we should get better predictions. Now, one thing to note is how the MSE values seem to be pretty high. This makes us wonder if there could be a better model that could predict the enrollment amount in a geographic region. However, based on our two candidate models, we can see that adding the division to our model will help us make predictions and thus, makes the multiple linear regression model a better choice over our simple linear regression model of just year.